In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pickle

In [24]:
# load the trained model, scaler pickle and one hot encoder pickle
model = tf.keras.models.load_model('model.h5')

# load the encoder and scaler
with open('one_hot_encoder_geography.pkl', 'rb') as file:
    one_hot_encoder_geography = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [25]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [36]:
input_df = pd.DataFrame([input_data])

In [37]:
geography_columns = one_hot_encoder_geography.columns.to_list()

geography_columns_df = { col: 0.0 for col in geography_columns }
geography_columns_df[f"Geography_{input_data['Geography']}"] = 1.0
geography_columns_df = pd.DataFrame([geography_columns_df])

input_df.Gender = label_encoder_gender.transform(input_df.Gender)
input_df = input_df.drop('Geography', axis=1).join(geography_columns_df)

In [38]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
# numpy array
input_df_array = input_df.to_numpy()

In [41]:
# apply scaling
input_df_array = scaler.transform(input_df_array)

c:\Users\MahaBelrhazi\anaconda3\envs\ann_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [43]:
input_df_array.shape

(1, 12)

In [45]:
model.predict(input_df_array)

1/1 [==============================] - 0s 20ms/step


array([[0.04543324]], dtype=float32)

In [47]:
prediction_proba = model.predict(input_df_array)[0][0] > 0.5

1/1 [==============================] - 0s 18ms/step


In [48]:
if prediction_proba > 0.5:
    print('The customer is likely to leave the bank')
else:
    print('The customer is likely to stay with the bank')

The customer is likely to stay with the bank
